In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix
import string
import matplotlib.pyplot as plt
import json
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout
from keras.optimizers import SGD
import random
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
import pickle
import warnings
warnings.filterwarnings('ignore')
import numpy as np

In [2]:
lemmatizer=WordNetLemmatizer()

In [3]:
intents=json.loads(open('intents.json').read())

In [4]:
words=[]
classes=[]
documents=[]
ignore_words=['.',',','?','!']

In [5]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list=nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list,intent['tag']))
        if intent['tag']not in classes:
            classes.append(intent['tag'])
print(documents)

[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['How', 'you', 'could', 'help', 'me', '?'], 'options'), (['What', 'you', 'can', 'do', '?'], 'options'), (['What', 'help', 'you', 'provide', '?'], 'options'), (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'), (['What', 'support', 'is', 'offered'], 'options'), (['How', 'to', 'check', 'Adverse', 'drug', 'reaction', '?'], 'adverse_drug'), (['Open', 'adverse', 'drugs', 'module'], 'adverse_drug'), (['Give', 'm

In [6]:
words=[lemmatizer.lemmatize(word) for word in words if words not in ignore_words]
words=sorted(set(words))
print(words)

["'s", ',', '?', 'Adverse', 'Awesome', 'Blood', 'Bye', 'Find', 'Give', 'Good', 'Goodbye', 'Hello', 'Hey', 'Hi', 'Hola', 'Hospital', 'How', 'I', 'ID', 'Is', 'List', 'Load', 'Locate', 'Looking', 'Lookup', 'Nice', 'Open', 'Search', 'Searching', 'See', 'Show', 'Task', 'Thank', 'Thanks', 'That', 'Till', 'What', 'Which', 'a', 'adverse', 'all', 'anyone', 'are', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'for', 'have', 'help', 'helpful', 'helping', 'history', 'hospital', 'is', 'later', 'list', 'log', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'of', 'offered', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'suitable', 'support', 'thanks', 'there', 'time', 'to', 'transfer', 'up', 'want', 'with', 'you']


In [7]:
classes=sorted(set(classes))

In [8]:
print(classes)

['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']


In [9]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [10]:
training=[]

In [11]:
output_empty=[0]*len(classes)

In [12]:
for document in documents:
    bag=[]
    word_patterns=document[0]
    word_patterns=[lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
        
    output_row=list(output_empty)
    output_row[classes.index(document[1])]=1
    training.append([bag,output_row])

In [13]:
random.shuffle(training)
training=np.array(training)

train_x=list(training[:, 0])
train_y=list(training[:, 1])

print('The training data is created')

The training data is created


In [14]:
model=Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))

sgd=SGD(lr=0.01,decay=1e-6,momentum=0.9,nesterov=True)
model.compile(loss='categorical_crossentropy',
             optimizer='SGD',
             metrics=['accuracy'])
hist=model.fit(np.array(train_x),np.array(train_y),epochs=200,batch_size=5,verbose=1)
model.save('chatbot_model.model',hist)
print("Done")

Epoch 1/200
10/10 [==============================] - 1s 4ms/step - loss: 2.2299 - accuracy: 0.1277
Epoch 2/200
10/10 [==============================] - 0s 4ms/step - loss: 2.1897 - accuracy: 0.1064
Epoch 3/200
10/10 [==============================] - 0s 4ms/step - loss: 2.1591 - accuracy: 0.1277
Epoch 4/200
10/10 [==============================] - 0s 3ms/step - loss: 2.1646 - accuracy: 0.1489
Epoch 5/200
10/10 [==============================] - 0s 4ms/step - loss: 2.1250 - accuracy: 0.2128
Epoch 6/200
10/10 [==============================] - 0s 4ms/step - loss: 2.1265 - accuracy: 0.2340
Epoch 7/200
10/10 [==============================] - 0s 4ms/step - loss: 2.1778 - accuracy: 0.2766
Epoch 8/200
10/10 [==============================] - 0s 3ms/step - loss: 2.1370 - accuracy: 0.2128
Epoch 9/200
10/10 [==============================] - 0s 5ms/step - loss: 2.1363 - accuracy: 0.1702
Epoch 10/200
10/10 [==============================] - 0s 3ms/step - loss: 2.1007 - accuracy: 0.2979
Epoch 11/

10/10 [==============================] - 0s 4ms/step - loss: 1.2438 - accuracy: 0.7021
Epoch 84/200
10/10 [==============================] - 0s 4ms/step - loss: 1.1646 - accuracy: 0.7660
Epoch 85/200
10/10 [==============================] - 0s 3ms/step - loss: 1.2174 - accuracy: 0.6596
Epoch 86/200
10/10 [==============================] - 0s 4ms/step - loss: 1.2116 - accuracy: 0.6596
Epoch 87/200
10/10 [==============================] - 0s 4ms/step - loss: 1.0579 - accuracy: 0.7660
Epoch 88/200
10/10 [==============================] - 0s 4ms/step - loss: 1.2496 - accuracy: 0.5532
Epoch 89/200
10/10 [==============================] - 0s 4ms/step - loss: 1.1191 - accuracy: 0.6809
Epoch 90/200
10/10 [==============================] - 0s 4ms/step - loss: 1.0205 - accuracy: 0.8085
Epoch 91/200
10/10 [==============================] - 0s 5ms/step - loss: 1.0551 - accuracy: 0.8085
Epoch 92/200
10/10 [==============================] - 0s 4ms/step - loss: 1.0687 - accuracy: 0.7234
Epoch 93/200


10/10 [==============================] - 0s 5ms/step - loss: 0.4959 - accuracy: 0.8723
Epoch 165/200
10/10 [==============================] - 0s 4ms/step - loss: 0.6444 - accuracy: 0.8511
Epoch 166/200
10/10 [==============================] - 0s 4ms/step - loss: 0.6494 - accuracy: 0.8511
Epoch 167/200
10/10 [==============================] - 0s 4ms/step - loss: 0.5941 - accuracy: 0.8511
Epoch 168/200
10/10 [==============================] - 0s 3ms/step - loss: 0.4634 - accuracy: 0.8936
Epoch 169/200
10/10 [==============================] - 0s 3ms/step - loss: 0.4653 - accuracy: 0.8723
Epoch 170/200
10/10 [==============================] - 0s 3ms/step - loss: 0.7026 - accuracy: 0.8511
Epoch 171/200
10/10 [==============================] - 0s 3ms/step - loss: 0.4827 - accuracy: 0.9362
Epoch 172/200
10/10 [==============================] - 0s 3ms/step - loss: 0.5797 - accuracy: 0.7872
Epoch 173/200
10/10 [==============================] - 0s 4ms/step - loss: 0.6020 - accuracy: 0.8723
Epoc

INFO:tensorflow:Assets written to: chatbot_model.model\assets


INFO:tensorflow:Assets written to: chatbot_model.model\assets


Done


In [15]:
from tensorflow.keras.models import load_model


In [ ]:
intents = json.loads(open('intents.json').read())
words=pickle.load(open('words.pkl','rb'))
classes=pickle.load(open('classes.pkl','rb'))
model=load_model('chatbot_model.model')

In [ ]:
def clean_up_sentence(sentence):
    sentence_words=nltk.word_tokenize(sentence)
    sentence_words=[lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

In [ ]:
def bag_of_words(sentence):
    sentence_words=clean_up_sentence(sentence)
    bag=[0]*len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word==w:
                bag[i]=1
    return np.array(bag)        

In [ ]:
def predict_class(sentence):
    bow=bag_of_words(sentence,)
    res=model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD=0.25
    result=[[i,r]for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    
    result.sort(key=lambda x: x[1],reverse=True)
    return_list=[]
    for r in result:
        return_list.append({'intent':classes[r[0]],'probability':str(r[1])})
        return return_list

In [ ]:
def get_response(intents_list,intents_json):
    tag=intents_list[0]['intent']
    list_of_intents=intents_json['intents']
    for i in list_of_intents:
        if i['tag']==tag:
            result=random.choice(i['responses'])
            break
    return result

print("Go! Bot is running!")

In [ ]:
while True:
    message=input("")
    ints=predict_class(message)
    res=get_response(ints,intents)
    print(res)

In [ ]:
#import tkinter
#from tkinter import *
#def send():
    #msg = EntryBox.get("1.0",'end-1c').strip()
    #EntryBox.delete("0.0",END)

    #if msg != '':
        #ChatBox.config(state=NORMAL)
        #ChatBox.insert(END, "You: " + msg + '\n\n')
        #ChatBox.config(foreground="#446665", font=("Verdana", 12 )) 
        #ints = predict_class(msg)
        #res = getResponse(ints, intents)
        
        #ChatBox.insert(END, "Bot: " + res + '\n\n')           
        #ChatBox.config(state=DISABLED)
        #ChatBox.yview(END)

#root = Tk()
#root.title("Chatbot")
#root.geometry("400x500")
#root.resizable(width=FALSE, height=FALSE)

#Create Chat window
#ChatBox = Text(root, bd=0, bg="white", height="8", width="50", font="Arial",)
#ChatBox.config(state=DISABLED)

#Bind scrollbar to Chat window
#scrollbar = Scrollbar(root, command=ChatBox.yview, cursor="heart")
#ChatBox['yscrollcommand'] = scrollbar.set

#Create Button to send message
#SendButton = Button(root, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    #bd=0, bg="#f9a602", activebackground="#3c9d9b",fg='#000000',

                    #command= send )
#Create the box to enter message
#EntryBox = Text(root, bd=0, bg="white",width="29", height="5", font="Arial")

#EntryBox.bind("<Return>", send)

#Place all components on the screen

#scrollbar.place(x=376,y=6, height=386)

#ChatBox.place(x=6,y=6, height=386, width=370)

#EntryBox.place(x=128, y=401, height=90, width=265)

#SendButton.place(x=6, y=401, height=90)


#root.mainloop()